In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore')

In [2]:
all_df = pd.read_csv('test1_modified_1.csv')
all_df.columns

Index(['대여일자', '대여시간', '대여구분코드', '성별', '연령대코드'], dtype='object')

### 데이터 통일
- 컬럼명 통일 후 컬럼 내의 데이터 점검
 - 데이터 형태 통일
 - 이상치 및 결측치 처리
 - 수치형 변환
- 데이터를 분석이 용이한 형태로 변환하는 것은 데이터 사이언티스트의 역량

#### 자료형 통일
- 대여구분코드
- 연령대코드

In [3]:
set(all_df['대여구분코드'])

# 중복이 제거된 데이터임에도 형태 통일 x
# 따옴표의 유무의 따라 두 가지 경우 존재
# 일일권, 정기권, 단체권 세 가지 유형으로 압축 가능

{"'단체'",
 "'일일(2시간권)'",
 "'일일(비회원)'",
 "'일일(회원)'",
 "'정기'",
 "'정기(2시간권)'",
 '단기권(전기)',
 '단체권',
 '일일권',
 '일일권(비회원)',
 '정기권'}

In [4]:
all_df['대여구분코드'].unique()

# set()과 마찬가지로 중복값을 제거하여 출력
# np.unique(all_df['대여구분코드']) -> NaN(float형) 존재할 경우 TypeError 발생, 느린 처리 속도

array(['정기권', '일일권', '단체권', '일일권(비회원)', '단기권(전기)', "'정기'", "'일일(회원)'",
       "'일일(비회원)'", "'단체'", "'일일(2시간권)'", "'정기(2시간권)'"], dtype=object)

In [5]:
set(all_df['연령대코드'])

# 나머지 값을 float형으로 변환하거나, nan을 string형으로 변환하여 연령대코드 데이터 통일

{"'20대'",
 "'30대'",
 "'40대'",
 "'50대'",
 "'60대'",
 "'70대~'",
 "'~10대'",
 '20대',
 '30대',
 '40대',
 '50대',
 '60대',
 '70대~',
 nan,
 '~10대'}

In [6]:
# 모든 데이터를 일괄적으로 string 변환
all_df['연령대코드'] = all_df['연령대코드'].apply(lambda x: str(x))
set(all_df['연령대코드'])

{"'20대'",
 "'30대'",
 "'40대'",
 "'50대'",
 "'60대'",
 "'70대~'",
 "'~10대'",
 '20대',
 '30대',
 '40대',
 '50대',
 '60대',
 '70대~',
 'nan',
 '~10대'}

#### 중복 따옴표 제거
- 대여구분코드, 연령대코드 컬럼에 따옴표가 두 번 사용된 데이터 존재
- 데이터를 문자열 형태로 만들어주는 큰 따옴표는 유지하고, 문자열 안의 불필요한 작은 따옴표 제거

In [7]:
# 따옴표가 없는 string 형태로 통일하기 위해 '' 제거
# '를 빈 문자열로 대체(replace) -> 이스케이프 문자 \ 사용
for col in all_df.columns:
    all_df[col] = all_df[col].apply(lambda x: str(x).replace('\'', ''))

In [8]:
set(all_df['연령대코드'])

{'20대', '30대', '40대', '50대', '60대', '70대~', 'nan', '~10대'}

#### 데이터 종류 재설정
- 대여구분코드
- 성별

In [9]:
##### 대여구분코드 #####

set(all_df['대여구분코드'])

# 11개의 경우의 수를 아래의 세 가지 유형으로 압축
# '단체*' -> group
# '단기*', '일일*' -> casual
# '정기*' -> regular

{'단기권(전기)',
 '단체',
 '단체권',
 '일일(2시간권)',
 '일일(비회원)',
 '일일(회원)',
 '일일권',
 '일일권(비회원)',
 '정기',
 '정기(2시간권)',
 '정기권'}

In [10]:
mapper = {'단기권(전기)':'casual',
          '단체':'group',
          '단체권':'group',
          '일일(2시간권)':'casual',
          '일일(비회원)':'casual',
          '일일(회원)':'casual',
          '일일권':'casual',
          '일일권(비회원)':'casual',
          '정기':'regular',
          '정기(2시간권)':'regular',
          '정기권':'regular'}

all_df['type'] = all_df['대여구분코드'].map(mapper)
set(all_df['type'])

{'casual', 'group', 'regular'}

In [11]:
all_df.head()

,대여일자,대여시간,대여구분코드,성별,연령대코드,type
0,2019-12-01,0,정기권,nan,~10대,regular
1,2019-12-01,0,정기권,nan,~10대,regular
2,2019-12-01,0,정기권,nan,~10대,regular
3,2019-12-01,0,정기권,nan,~10대,regular
4,2019-12-01,0,정기권,nan,~10대,regular


In [12]:
##### 성별 #####

np.unique(all_df['성별'])

array(['', 'F', 'M', '\\N', 'f', 'm', 'nan'], dtype=object)

In [13]:
mapper = {'':'U', # Unidentified
          'F':'F', # Female
          'M':'M', # Male
          '\\N':'U',
          'f':'F',
          'm':'M',
          'nan':'U'}

all_df['gender'] = all_df['성별'].map(mapper)
set(all_df['gender'])

{'F', 'M', 'U'}

In [14]:
all_df.head()

,대여일자,대여시간,대여구분코드,성별,연령대코드,type,gender
0,2019-12-01,0,정기권,nan,~10대,regular,U
1,2019-12-01,0,정기권,nan,~10대,regular,U
2,2019-12-01,0,정기권,nan,~10대,regular,U
3,2019-12-01,0,정기권,nan,~10대,regular,U
4,2019-12-01,0,정기권,nan,~10대,regular,U


#### 시간 정보 처리
- 대여일자 및 대여시간
- 새로운 형식의 datetime 컬럼 생성

In [15]:
# 대여일자와 대여시간을 이용하여 '연-월-일 시:분:초' 형식의 datetime 생성
# ex) 2019-12-01 0 -> 2019-12-01 00:00:00

In [16]:
# 데이터 타입 확인
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45108029 entries, 0 to 45108028
Data columns (total 7 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   대여일자    object
 1   대여시간    object
 2   대여구분코드  object
 3   성별      object
 4   연령대코드   object
 5   type    object
 6   gender  object
dtypes: object(7)
memory usage: 2.4+ GB


#### 대여시간

In [17]:
all_df.head()

,대여일자,대여시간,대여구분코드,성별,연령대코드,type,gender
0,2019-12-01,0,정기권,nan,~10대,regular,U
1,2019-12-01,0,정기권,nan,~10대,regular,U
2,2019-12-01,0,정기권,nan,~10대,regular,U
3,2019-12-01,0,정기권,nan,~10대,regular,U
4,2019-12-01,0,정기권,nan,~10대,regular,U


In [18]:
set(all_df['대여시간'])

# 대여시간 컬럼에는 분, 초를 제외한 시 정보만 존재
# 또한, 자전거를 대여한 시간대의 분, 초는 중요하지 x -> 판단을 위한 도메인 지식 필요
# 따라서 모든 대여시간 데이터를 '00' 형태로 변환한 후 뒤에 ':00:00'을 추가

{'0',
 '00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9'}

In [19]:
# 대여시간 정보를 '00' 형태로 변환하는 샘플 연산
sample_date = '2019-12-01'
sample_hour = '0'

print(sample_date + ' ' + sample_hour.zfill(2) + ':00:00') # zfill() 사용
print(sample_date + ' ' + str('0' + sample_hour)[-2:] + ':00:00') # 맨 앞에 0 삽입 -> 마지막 두 문자만 추출

2019-12-01 00:00:00
2019-12-01 00:00:00


In [20]:
# all_df['대여시간'] = all_df['대여시간'].apply(lambda x: str('0' + x)[-2:])
all_df['대여시간'] = all_df['대여시간'].apply(lambda x: x.zfill(2))
set(all_df['대여시간'])

{'00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23'}

#### 대여일자

In [21]:
all_df

# 대여일자 데이터에 일관성 x

,대여일자,대여시간,대여구분코드,성별,연령대코드,type,gender
0,2019-12-01,00,정기권,nan,~10대,regular,U
1,2019-12-01,00,정기권,nan,~10대,regular,U
2,2019-12-01,00,정기권,nan,~10대,regular,U
3,2019-12-01,00,정기권,nan,~10대,regular,U
4,2019-12-01,00,정기권,nan,~10대,regular,U
...,...,...,...,...,...,...,...
45108024,2018-11-30 00:00:00,23,일일권,M,30대,casual,M
45108025,2018-11-30 00:00:00,23,일일권,M,30대,casual,M
45108026,2018-11-30 00:00:00,23,일일권,M,40대,casual,M
45108027,2018-11-30 00:00:00,23,일일권,M,60대,casual,M


In [22]:
# 길이 정보로 대여일자 데이터의 형태 확인
set(all_df['대여일자'].apply(lambda x: len(x)))

# 어떤 데이터는 10자리, 나머지는 19자리
# 10자리로 통일 필요

{10, 19}

In [23]:
# 앞에서부터 10자리만 추출
all_df['대여일자'] = all_df['대여일자'].apply(lambda x: x[:10])
set(all_df['대여일자'].apply(lambda x: len(x)))

{10}

In [24]:
# 10자리가 모두 연-월-일 형태로 이루어져 있는지 확인
print(set(all_df['대여일자'].apply(lambda x: x[:4])))
print(set(all_df['대여일자'].apply(lambda x: x[5:7])))

{'2017', '2019', '2021', '2018', '2020'}
{'07', '12', '09', '01', '08', '02', '06', '11', '10', '04', '05', '03'}


#### datetime 컬럼 생성

In [25]:
all_df['datetime'] = all_df['대여일자'] + ' ' + all_df['대여시간'] + ':00:00'
all_df['datetime']

0           2019-12-01 00:00:00
1           2019-12-01 00:00:00
2           2019-12-01 00:00:00
3           2019-12-01 00:00:00
4           2019-12-01 00:00:00
                   ...         
45108024    2018-11-30 23:00:00
45108025    2018-11-30 23:00:00
45108026    2018-11-30 23:00:00
45108027    2018-11-30 23:00:00
45108028    2018-11-30 23:00:00
Name: datetime, Length: 45108029, dtype: object

#### 연령대코드 전처리
- 데이터 이름 수정(형식 통일)
 - 나이로 연산할 것이 아니므로 문자열 형태 유지

In [26]:
all_df['연령대코드'].unique()

array(['~10대', '20대', '30대', '40대', '50대', '60대', '70대~', 'nan'],
      dtype=object)

In [27]:
mapper = {'20대':'20s',
          '30대':'30s',
          '40대':'40s',
          '50대':'50s',
          '60대':'60s',
          '70대~':'70s',
          'nan':'unknown',
          '~10대':'10s'}

all_df['age'] = all_df['연령대코드'].map(mapper)
all_df.head()

,대여일자,대여시간,대여구분코드,성별,연령대코드,type,gender,datetime,age
0,2019-12-01,00,정기권,nan,~10대,regular,U,2019-12-01 00:00:00,10s
1,2019-12-01,00,정기권,nan,~10대,regular,U,2019-12-01 00:00:00,10s
2,2019-12-01,00,정기권,nan,~10대,regular,U,2019-12-01 00:00:00,10s
3,2019-12-01,00,정기권,nan,~10대,regular,U,2019-12-01 00:00:00,10s
4,2019-12-01,00,정기권,nan,~10대,regular,U,2019-12-01 00:00:00,10s


#### Feature/Label 구분
- type, gender, datetime, age 컬럼을 구분
 - datetime은 feature, type/gender/age은 label에 가까운 변수
 - ex) gender가 label이라면 성별에 따른 이용량, age가 label이라면 나이에 따른 이용량

In [28]:
all_df.columns

Index(['대여일자', '대여시간', '대여구분코드', '성별', '연령대코드', 'type', 'gender', 'datetime',
       'age'],
      dtype='object')

In [29]:
# 분석에 사용할 컬럼 목록
selected_columns = ['datetime', 'type', 'gender', 'age']

df = all_df[selected_columns]
df.head()

,datetime,type,gender,age
0,2019-12-01 00:00:00,regular,U,10s
1,2019-12-01 00:00:00,regular,U,10s
2,2019-12-01 00:00:00,regular,U,10s
3,2019-12-01 00:00:00,regular,U,10s
4,2019-12-01 00:00:00,regular,U,10s


In [30]:
# 체크 포인트: 분석에 사용할 데이터를 csv 파일로 저장
df.to_csv('test1_modified_2.csv', index=False)